**FidlTrack Ambiguity** allows to detect, quantify and remove ambiguous displacements from SPT trajectory data.

Note: this script can only recompute ambiguity based on Euclidean distance. To get the ambiguity based on graph distance, use the structure-aware tracking extension from Trackmate with the EdgeAmbiguityAnalyzer.

The pre-filled parameters correspond to the files obtained by tracking with the provided FidlTrack scripts.

**Setup**

In [ ]:
#@markdown Launch this cell to initialise the notebook (you only need to run this once)
import os
from google.colab import output

import numpy as np
from matplotlib import pyplot as plt
from matplotlib.patches import Rectangle
import matplotlib as mpl

is_dark = output.eval_js('document.documentElement.matches("[theme=dark]")')
print(is_dark)

base_path = os.getcwd()

**Link google drive**

In [ ]:
#@markdown Give access to your Google Drive (you only need to run this once)

from google.colab import drive
drive.mount(base_path + '/gdrive')

**Compute Ambiguity**

The files need to be in your google drive. To find the correct path to your files, go to the Files menu on the left of the notebook (folder icon), navigate to your file, right-click on it and choose "Copy path", and paste the result into the textbox below.

In [ ]:
def pos_to_gpos(p, minp, dx):
  return np.floor((p - minp) / dx).astype("int")

def gpos_to_pos(idx, minp, dx):
  return minp + idx * dx

#INPUTS
#@markdown Outputs
show_ambiguous_disps = True # @param {type:"boolean"}
show_ambiguities_time = False # @param {type:"boolean"}
show_ambig_map = False # @param {type:"boolean"}
ambig_map_dx = 0.5#@param {type:"number"}

#@markdown Load Trajectories
traj_path = ""#@param {type:"string"}
link_dist = 1.0#@param {type:"raw"}
#@markdown Trajectory parsing
traj_header = 1#@param {type:"integer"}
separator_trajs = ","#@param {type:"string"}
traj_id_col = 0#@param {type:"integer"}
traj_frame_col = 5#@param {type:"integer"}
traj_xpos_col = 2#@param {type:"integer"}
traj_ypos_col = 3#@param {type:"integer"}

#@markdown Are ambiguities already computed in the trajectory file or do we need to recompute them from the spots?
getAmbiguityFrom = 'trajectories' # @param ["trajectories", "spots"]
#@markdown Column containing ambiguity information in trajectory file - fill only if using ambiguity from trajectories (starts at 0)
ambig_col = 6#@param {type:"integer"}
#@markdown Load spots file to compute ambiguity - fill only if using ambiguity from spots
spot_path = ""#@param {type:"string"}
spot_header = 1#@param {type:"integer"}
separator_spot = ","#@param {type:"string"}
spt_frame_col = 3#@param {type:"integer"}
spt_xpos_col = 1#@param {type:"integer"}
spt_ypos_col = 2#@param {type:"integer"

#PARAMETERS CONVERTION
if link_dist is not None:
  link_dist = float(link_dist)

ok = True
if not os.path.exists(traj_path):
  print("File not found:" + traj_path)
  ok = False

if ok:
  trajs = []
  with open(traj_path, 'r') as f:
    i = 0
    for line in f:
      i+= 1
      if i <= traj_header:
        continue
      l = line.rstrip("\n").split(separator_trajs)
      tmp = [int(l[traj_id_col]), int(float(l[traj_frame_col])), float(l[traj_xpos_col]), float(l[traj_ypos_col])]
      if getAmbiguityFrom == "trajectories":
        tmp.append(int(float(l[ambig_col])))
      trajs.append(tmp)

  if getAmbiguityFrom == "trajectories":
    tab = np.array(trajs)

  if getAmbiguityFrom == "spots":
    tab = np.ones((len(trajs), 5)) * -1
    tab[:,0:4] = np.array(trajs)

    spts = []
    with open(spot_path, 'r') as f:
      i = 0
      for line in f:
        i += 1
        if i <= spot_header:
          continue
        l = line.rstrip("\n").split(separator_spot)
        spts.append([int(float(l[spt_frame_col])), float(l[spt_xpos_col]), float(l[spt_ypos_col])])
    spts = np.array(spts)

    for i in range(tab.shape[0]-1):
      if tab[i,0] == tab[i+1,0]:
        succs = spts[spts[:,0] == tab[i+1,1],:]

        cnt = np.sum(np.sum((succs[:,1:3] - tab[i,2:4])**2, axis=1) <= link_dist**2)
        assert(cnt > 0)
        tab[i,4] = cnt - 1

  nambigs = np.sum(tab[:,4] > 0)
  ndisps = np.sum([np.sum(tab[:,0] == i) - 1 for i in set(tab[:,0])])
  print("Ambiguity Score = {:.1f} %".format((nambigs / ndisps) * 100))

  if show_ambiguous_disps:
    plt.figure(figsize=(7,7))
    for i in np.where(tab[:,4] == 0)[0]: #non-ambiguous disps
      plt.plot(tab[i:i+2, 2], tab[i:i+2, 3], 'k')
    for i in np.where(tab[:,4] > 0)[0]: #ambiguous disps
      plt.plot(tab[i:i+2, 2], tab[i:i+2, 3], 'r')
    plt.xlabel("x-postion")
    plt.ylabel("y-postion")

  if show_ambiguities_time:
    ambig_time = np.zeros((int(np.max(tab[:,1]))+1, 1))
    for i in np.where(tab[:,4] > 0)[0]:
      ambig_time[int(tab[i,1])] += 1

    plt.figure(figsize=(7,7))
    plt.step(range(int(np.max(tab[:,1]))+1), ambig_time)
    plt.xlabel("Frame")
    plt.ylabel("Number of ambiguous displacements")

  if show_ambig_map:
    minp = np.array([0, 0])
    maxp = np.max(tab[:, 2:4], axis=0)

    amap = {}
    for i in np.where(tab[:,4] > 0)[0]:
      idx = pos_to_gpos(tab[i, 2:4], minp, ambig_map_dx)
      amap.setdefault(idx[0], {})
      amap[idx[0]].setdefault(idx[1], 0)
      amap[idx[0]][idx[1]] += 1.0

    cmap = mpl.colormaps['viridis']
    norm = mpl.colors.Normalize(vmin=0,
                                vmax=max(max([v for v in amap[k].values()]) for k in amap.keys()))

    fig, ax = plt.subplots(1)
    fig.figsize = (7,7)
    for i1 in amap.keys():
      for i2 in amap[i1].keys():
        idx = np.array([i1, i2])
        pos = gpos_to_pos(idx, minp, ambig_map_dx)
        ax.add_patch(Rectangle(pos, ambig_map_dx, ambig_map_dx,
                              facecolor=cmap(norm(amap[i1][i2]))))
    plt.axis([minp[0], maxp[0], minp[1], maxp[1]])
    plt.colorbar(mpl.cm.ScalarMappable(cmap=cmap, norm=norm),
                label="Number of ambiguous displacements", ax=ax)
    plt.xlabel("x-postion")
    plt.ylabel("y-postion")

**Remove Ambiguous Displacements**

Split trajectories around ambiguous displacements generating two sub-trajectories (containing all displacements before / after the ambiguous displacement).

This cell depend on the ambiguity detection done in the previous cell.

This cell generates a file named traj_path + "_noa.csv", where traj_path is the value from the previous cell, containing ambiguity-free trajectories.

In [ ]:
show_trajs_rm_ambig = False # @param {type:"boolean"}

#Split trajs around ambiguous displacements and save the result
tab_noa = []
with open(traj_path + "_noa.csv", "w") as f:
  M = 1
  for i in range(tab.shape[0]):
    if i > 0 and tab[i-1,0] != tab[i,0] or tab[i,4] > 0:
      M += 1
    tab_noa.append([M, tab[i,1], tab[i,2], tab[i,3], tab[i,4]])
    f.write(separator_trajs.join([str(e) for e in tab_noa[-1]]) + "\n")
tab_noa = np.array(tab_noa)

print("Num. trajs.: {} raw -> {} without ambigs".format(
    len(set(tab[:,0])), len(set(tab_noa[:,0]))))
print("AVG±SD traj. len (num spots): {:.1f} ± {:.1f} raw -> {:.1f} ± {:.1f} without ambigs".format(
    np.mean([np.sum(tab[:,0]==i) for i in set(tab[:,0])]),
    np.std([np.sum(tab[:,0]==i) for i in set(tab[:,0])]),
    np.mean([np.sum(tab_noa[:,0]==i) for i in set(tab_noa[:,0])]),
    np.std([np.sum(tab_noa[:,0]==i) for i in set(tab_noa[:,0])])))

if show_trajs_rm_ambig:
  plt.figure(figsize=(7,7))
  plt.title("Trajectories after removal of ambiguous displacements")
  for i in set(tab_noa[:,0]):
    plt.plot(tab_noa[tab_noa[:,0]==i, 2], tab_noa[tab_noa[:,0]==i, 3],
              color=np.random.rand(3))
  plt.xlabel("x-postion")
  plt.ylabel("y-postion")